<left>
    <a href=img><img src = "https://github.com/tiagottmoraes/CS-s-Data-Science-Test/blob/main/Misc/1c86bfb3-cab0-4255-99bf-24c5488d888a-1632765661247.png?raw=true" width="400"  />
</left>

###### Developed by Tiago Moraes (09.02.2022)

# Data Science Test - Part I - Intro

----

The following notebooks presents the complete dataset analysis, exploration and pre-processing steps required to implement a tailored machine learning model designed to **predict the total minutes** ('total_minutes' label) it takes to a shopper to complete a given order.

For that, **3 different notebooks** were prepared for each task, as following:

- **[1. ETL process notebook:](#https://github.com/tiagottmoraes/CS-s-Data-Science-Test/blob/main/1.ETL_-_Data_Science_Test.ipynb)** Complete ETL (data wrangling) process of the provided datasets, including feature engineering, feature transformations, and data standardizations, necessary to carry out all the statistics tests necessaries in the next phase;

- **[2. EDA notebook:](#https://github.com/tiagottmoraes/CS-s-Data-Science-Test/blob/main/2.EDA_-_Data_Science_Test.ipynb)** Statistical exploration and understanding of the pre-processed dataset. Needed to define the best machine learning strategy for the target Label;

- **[3. Machine Learning Model development notebook:](#https://github.com/tiagottmoraes/CS-s-Data-Science-Test/blob/main/3.ML_Model_-_Data_Science_Test.ipynb)** Build, train and test a ML model that takes in the process the dataset, generates a model and deploy it with unseen data to generate predictions ("total_minutes" label)
<br />
<br />

For this test goal, 4 different datasets were provided, each of them with different types of features and volume of data. 

**order_products.csv (198500 rows)**

*order_id*: ID of the order<br />
*product_id*: ID of the product<br />
*quantity*: The quantity ordered of this product<br />
*buy_unit*: The unit of the product (KG/UN)

**orders.csv (10000 rows)**

*order_id*: ID of the order<br />
*lat*: The latitude of the delivery location<br />
*lng*: The longitude of the delivery location<br />
*promised_time*: The delivery time promised to the user<br />
*on_demand*: If true, the order was promised to be delivered in less than X minutes<br />
*shopper_id*: ID representing the shopper completed the order<br />
*store_branch_id*: ID of the store branch<br />
***total_minutes***: The total minutes it took to complete the order (label)

**shopper.csv (2864 rows)**

*shopper_id*: ID of the shopper<br />
*seniority*: The experience level of the shopper<br />
*found_rate*: Percentage of products found by shopper historical<br />
*picking_speed*: Historical picking speed, products pr minutes<br />
*accepted_rate*: Percentage of orders historically accepted by shopper<br />
*rating*: client rating of shopper

**storebranch.csv (476 rows)**

*store_branch_id*: ID of the store branch<br />
*store*: ID representing the store<br />
*lat*: Latitude of the branch location<br />
*lng*: Longitude of the branch location
<br />
<br />


# 1. ETL process - Data wrangling, featute engineering and data standardization


----

## 1. Importing necessary modules

In [113]:
import pandas as pd
import numpy as np
from geopy import Point
from geopy.distance import distance

#### 1.1 Understanding primary keys between tables

Before start loading the datasets, let's take a look at how the tables are connected to each other and have a better undestanding of case at hand.



<center>
    <a href=img><img src = "https://github.com/tiagottmoraes/CS-s-Data-Science-Test/blob/main/entity_mapping.PNG?raw=true" width="800"  />
</center>

As we can see from the entity map above, primary keys from "storebranch.csv", "order_product.csv" , "shopper_ID.csv" can be found in the "orders.csv", which happens to be the table where the label ("total_minutes") is located. In this case, the better approach would be to unify (merge) all tables together, using the primary keys, into the table "order.csv". 

## 2. Importing datasets

### 2.1 Loading dataset "order_prod.csv"

After loading the order_prod.csv into a dataframe called **df_ordprod**, let's look at the first five rows using the `head()` function:

In [114]:
## Order_products
filename_ordprod = "https://raw.githubusercontent.com/tiagottmoraes/CS-s-Data-Science-Test/main/data/order_products.csv"
df_ordprod = pd.read_csv(filename_ordprod)
df_ordprod.head()


,order_id,product_id,quantity,buy_unit
0,47099653730fb1b76537fc10ad876255,c1244453d731c77416cb4766e3bd76cb,1.0,UN
1,689d8866915acf87e851c2591a23a82f,43cc2b100bec640fe563cd16f2db669f,1.0,KG
2,f26d16bf6f38c9e31d0be877f4013a9e,b8f880759d014134e272d881d49989a2,1.0,UN
3,161ccc896835ab41761b0e726becb6b1,dbc062b9bef805d27a6f4bea7edfe1f1,1.0,UN
4,4713deca10bb5db98fae150b52d61fc0,93a060f269bb569398921100f84c519a,2.0,UN


Checking missing data in df_ordprod:

In [115]:
missing_data_order = df_ordprod.isnull()
for column in missing_data_order.columns.values.tolist():
    print(column)
    print(missing_data_order[column].value_counts())
    print("")

order_id
False    198500
Name: order_id, dtype: int64

product_id
False    198500
Name: product_id, dtype: int64

quantity
False    198500
Name: quantity, dtype: int64

buy_unit
False    198500
Name: buy_unit, dtype: int64



NOTE: No missing values were found within this dataframe 

2.1.1 Feature engineering

In [116]:
print('Unique products listed:', df_ordprod['product_id'].nunique(),'\n'
      'Unique orders listed:', df_ordprod['order_id'].nunique())

Unique products listed: 15422 
Unique orders listed: 9978


The number of products is much bigger than the number of orders made, which is to be expected, since a single order takes many products. With that prospect in mind, an assumption made from now on is that, **what defines the complexity of a given order is the number of unique products, not the the quantity of each product**.<br />
<br />
A simple example that illustrates this: an order consisting of 12 different products (one item per product) demands much more attention and movement (and most likely *time*) from a shopper than what an order consisting of only 3 products, but with several items per product would.<br />
<br />
With that in mind, the "df_ordprod" was _grouped by the order_ID feature_, since that will give us the number of unique products present in each order, hence, giving us the indicator that is most likely linked to the complexity of a given order (which, in turn, makes orders take more time to be completed).

In [117]:
df_ordprod = df_ordprod.groupby(["order_id"], as_index=False).count()
df_ordprod.rename(columns={'product_id':'order_size'}, inplace=True)
df_ordprod.head()

,order_id,order_size,quantity,buy_unit
0,0004a3841c1eeb6c6e77585a941c21e0,4,4,4
1,0005a6ecbbde1e8d273f5577bcff2c9c,1,1,1
2,0007baeb6700fc203be2d1f1e11222d7,22,22,22
3,0012195a6a8ca9ec308a3010eeea8ebc,11,11,11
4,0013011fa72b498b9feb84f4e7104980,44,44,44


NOTE: "product_id" was renamed to "order_size", since it better represents the number of unique products (list of items) contained in each order

### 2.2 Loading dataset "shopper.csv"

After loading the shoppers.csv into a dataframe called **df_shop**, let's look at the first five rows using the `head()` function:

In [118]:
## Shopper
filename_shop = "https://raw.githubusercontent.com/tiagottmoraes/CS-s-Data-Science-Test/main/data/shoppers.csv"
df_shop = pd.read_csv(filename_shop)
df_shop.head()

,shopper_id,seniority,found_rate,picking_speed,accepted_rate,rating
0,1fc20b0bdf697ac13dd6a15cbd2fe60a,41dc7c9e385c4d2b6c1f7836973951bf,0.8606,1.94,1.00,4.87
1,e1c679ac73a69c01981fdd3c5ab8beda,6c90661e6d2c7579f5ce337c3391dbb9,0.8446,1.23,0.92,4.92
2,09d369c66ca86ebeffacb133410c5ee1,6c90661e6d2c7579f5ce337c3391dbb9,0.8559,1.56,1.00,4.88
3,db39866e62b95bb04ebb1e470f2d1347,50e13ee63f086c2fe84229348bc91b5b,NaN,2.41,NaN,NaN
4,8efbc238660053b19f00ca431144fdae,6c90661e6d2c7579f5ce337c3391dbb9,0.8770,1.31,0.92,4.88


Checking missing data in df_shop:

In [119]:
missing_data_order = df_shop.isnull()
for column in missing_data_order.columns.values.tolist():
    print(column)
    print(missing_data_order[column].value_counts())
    print("")

shopper_id
False    2864
Name: shopper_id, dtype: int64

seniority
False    2864
Name: seniority, dtype: int64

found_rate
False    2763
True      101
Name: found_rate, dtype: int64

picking_speed
False    2864
Name: picking_speed, dtype: int64

accepted_rate
False    2837
True       27
Name: accepted_rate, dtype: int64

rating
False    2780
True       84
Name: rating, dtype: int64



NOTE: There were found:
- 101 NaN elements in the 'found_rate' feature
- 27 NaN elements in the 'accepted_rate' feature
- 84 NaN elements in the 'rating' feature

In [120]:
print('Unique shoppers listed:', df_shop['shopper_id'].nunique(),'\n' 
      'Unique seniority listed:', df_shop['seniority'].nunique())

Unique shoppers listed: 2864 
Unique seniority listed: 4


Despite being anonymized, the "seniority" feature consists of only 4 different classes. In order to help clarify this feature and its relationship with other elements in this dataframe, the original hash code was replaced by random letters (T, I, M, W) and had its data type replaced by categorical.

In [121]:
df_shop['seniority'].unique()

array(['41dc7c9e385c4d2b6c1f7836973951bf',
       '6c90661e6d2c7579f5ce337c3391dbb9',
       '50e13ee63f086c2fe84229348bc91b5b',
       'bb29b8d0d196b5db5a5350e5e3ae2b1f'], dtype=object)

In [122]:
df_shop.replace('41dc7c9e385c4d2b6c1f7836973951bf', 'T', inplace=True)
df_shop.replace('6c90661e6d2c7579f5ce337c3391dbb9','I', inplace=True)
df_shop.replace('50e13ee63f086c2fe84229348bc91b5b','M', inplace=True)
df_shop.replace('bb29b8d0d196b5db5a5350e5e3ae2b1f','W', inplace=True)
df_shop ['seniority']= df_shop['seniority'].astype('category')

In [123]:
# Counting the number of shoppers in each seniority class
df_shop['seniority'].value_counts()

I    1643
M     719
T     440
W      62
Name: seniority, dtype: int64

In [124]:
df_shop.sample(2)

,shopper_id,seniority,found_rate,picking_speed,accepted_rate,rating
158,39fb50af82e35b80a5030fee42db659a,M,0.8409,1.60,0.84,4.88
2643,8b1877e4569dacb372c35207b546f8d2,M,0.8249,2.03,0.80,4.84


### 2.3 Loading dataset "storebranch.csv"

After loading the storebranch.csv into a dataframe called **df_store**, let's look at the first five rows using the `head()` function:

In [125]:
## Storebranch
filename_store = "https://raw.githubusercontent.com/tiagottmoraes/CS-s-Data-Science-Test/main/data/storebranch.csv"
df_store = pd.read_csv(filename_store)
df_store.head()

,store_branch_id,store_id,lat,lng
0,aff1621254f7c1be92f64550478c56e6,92cc227532d17e56e07902b254dfad10,-33.422497,-70.609231
1,56352739f59643540a3a6e16985f62c7,0336dcbab05b9d5ad24f4333c7658a0e,-33.385484,-70.555579
2,7d04bbbe5494ae9d2f5a76aa1c00fa2f,9bf31c7ff062936a96d3c8bd1f8f2ff3,-33.416579,-70.565224
3,2b24d495052a8ce66358eb576b8912c8,c4ca4238a0b923820dcc509a6f75849b,-33.512578,-70.655952
4,5487315b1286f907165907aa8fc96619,d82c8d1619ad8176d665453cfb2e55f0,-33.347645,-70.542229


Checking missing data in df_store:

In [126]:
missing_data_order = df_store.isnull()
for column in missing_data_order.columns.values.tolist():
    print(column)
    print(missing_data_order[column].value_counts())
    print("")

store_branch_id
False    476
Name: store_branch_id, dtype: int64

store_id
False    476
Name: store_id, dtype: int64

lat
False    476
Name: lat, dtype: int64

lng
False    476
Name: lng, dtype: int64



NOTE: No missing values were found within this dataframe 

In [127]:
print('Unique store_branch listed:', df_store['store_branch_id'].nunique(),'\n' 'Unique store listed:', df_store['store_id'].nunique())

Unique store_branch listed: 476 
Unique store listed: 221


Considering that both "storebranch.csv" and "orders.csv" have different data on latitude and longitude coordinates but uses the same name convention, let's change both column's names to help differentiate them

In [128]:
# renaming the columns
df_store = df_store.rename(columns={'lat':'store_lat', 'lng':'store_long'})

### 2.4 Loading dataset "orders.csv"

After loading the orders.csv into a dataframe called **df_order**, let's look at the first five rows using the `head()` function:

In [129]:
## ORDERS - LABEL
filename_ORDER = "https://raw.githubusercontent.com/tiagottmoraes/CS-s-Data-Science-Test/main/data/orders.csv"
df_ORDER = pd.read_csv(filename_ORDER)
df_ORDER.head()

,order_id,lat,lng,promised_time,on_demand,shopper_id,store_branch_id,total_minutes
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,2019-10-18 20:48:00+00:00,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,2019-10-19 01:00:00+00:00,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,2019-10-19 14:54:00+00:00,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN
3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,2019-10-18 21:47:00+00:00,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742
4,b4b2682d77118155fe4716300ccf7f39,-33.403239,-70.564020,2019-10-19 20:00:00+00:00,False,5c5199ce02f7b77caa9c2590a39ad27d,1f0e3dad99908345f7439f8ffabdffc4,140.724822


Checking missing data in df_ORDER:

In [130]:
missing_data_order = df_ORDER.isnull()
for column in missing_data_order.columns.values.tolist():
    print(column)
    print(missing_data_order[column].value_counts())
    print("")

order_id
False    10000
Name: order_id, dtype: int64

lat
False    10000
Name: lat, dtype: int64

lng
False    10000
Name: lng, dtype: int64

promised_time
False    10000
Name: promised_time, dtype: int64

on_demand
False    10000
Name: on_demand, dtype: int64

shopper_id
False    10000
Name: shopper_id, dtype: int64

store_branch_id
False    10000
Name: store_branch_id, dtype: int64

total_minutes
False    8000
True     2000
Name: total_minutes, dtype: int64



NOTE: No null values, EXCEPT for the label "total_minutes". The rows containg those missing values in the label will be later set apart to be used during the predictions phase.

Considering that both "storebranch.csv" and "orders.csv" have different data on latitude and longitude coordinates but uses the same name convention, let's change both column's names to help differentiate them.

In [131]:
df_ORDER = df_ORDER.rename(columns={'lat':'delivery_lat', 'lng':'delivery_long'})

### 2.5 Merging dataframes

Merging dataframes using primary and foreign keys.

1. First, merge df_ordprod in df_ORDER, using "order_id" as primary keys:

In [132]:
df_merge = pd.merge(df_ORDER, df_ordprod, on='order_id')

2. Then, merge df_shop in df_merge (result from previous merge operation), using "shopper_id" as foreing keys:

In [133]:
df_merge = pd.merge(df_merge, df_shop, on='shopper_id')

3. Finally, merge df_store in df_merge (result from previous merge operation), using "store_branch_id" as foreing keys:

In [134]:
df_merge = pd.merge(df_merge, df_store, on='store_branch_id')

After merging and, then, sorting the features, we get the following dataframe:

In [135]:
df_merge = df_merge.reindex(['order_id','store_branch_id','store_lat','delivery_lat','store_long','delivery_long','promised_time', 'store_id','quantity', 'buy_unit','shopper_id','on_demand','seniority', 'order_size', 'found_rate','picking_speed','accepted_rate','rating', 'total_minutes'], axis=1)
pd.set_option('display.max_columns', None)
df_merge.sample(2)

,order_id,store_branch_id,store_lat,delivery_lat,store_long,delivery_long,promised_time,store_id,quantity,buy_unit,shopper_id,on_demand,seniority,order_size,found_rate,picking_speed,accepted_rate,rating,total_minutes
9707,99d820553b0d65d997f2af149348eb7b,a981f2b708044d6fb4a71a1463242520,-33.516349,-33.549886,-70.708798,-70.678444,2019-10-19 16:39:00+00:00,c4ca4238a0b923820dcc509a6f75849b,9,9,8b2cb89d01f000f9442a6cfcf9d73298,True,T,9,0.8325,2.11,0.96,4.92,83.259952
4933,55051b87f4345867316d69ae15eb4984,c4ca4238a0b923820dcc509a6f75849b,-33.402024,-33.390157,-70.516727,-70.508498,2019-10-19 21:00:00+00:00,c4ca4238a0b923820dcc509a6f75849b,29,29,23699070639432a286d3c5d7a144ada2,False,W,29,NaN,2.19,1.00,NaN,73.818684


## 3 Dealing with missing data

### 3.1 Removing/Replacing missing values

First, let's remove some unnecessary features, such as "promissed_time" (nothing to compare with), "store_id" (already used store_branch_id as primary key),"quantity" and "buy_unit" (both deprecated after grouping by "order_id", as show in item 2.1)

In [136]:
df_merge.drop(['promised_time', 'store_id','quantity', 'buy_unit'], axis=1, inplace=True)

As shown before, some features presents missing values that must be addressed before we head to the exploratory analysis. 


In [137]:
df_merge.sample(2)

,order_id,store_branch_id,store_lat,delivery_lat,store_long,delivery_long,shopper_id,on_demand,seniority,order_size,found_rate,picking_speed,accepted_rate,rating,total_minutes
9395,17ac6497fc7cf5b16ea9705a1eaea3a9,8e296a067a37563370ded05f5a3bf3ec,-33.385118,-33.375417,-70.574506,-70.547071,ee0b0c0b71cb4eb1ec5b0ea13db534fe,True,M,3,0.7895,2.71,1.00,4.93,35.169972
7114,ce6d073d3a783071040db2f5f7c16f86,e4bb4c5173c2ce17fd8fcd40041c068f,-33.484136,-33.447919,-70.621945,-70.641205,2f6231b5d7fc059c836b551a3d17ba7b,False,I,43,0.8530,1.15,0.92,4.68,162.908272


In [138]:
#checking the overall distribution and volume of the dataframe
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9978 entries, 0 to 9977
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   order_id         9978 non-null   object  
 1   store_branch_id  9978 non-null   object  
 2   store_lat        9978 non-null   float64 
 3   delivery_lat     9978 non-null   float64 
 4   store_long       9978 non-null   float64 
 5   delivery_long    9978 non-null   float64 
 6   shopper_id       9978 non-null   object  
 7   on_demand        9978 non-null   bool    
 8   seniority        9978 non-null   category
 9   order_size       9978 non-null   int64   
 10  found_rate       9779 non-null   float64 
 11  picking_speed    9978 non-null   float64 
 12  accepted_rate    9932 non-null   float64 
 13  rating           9816 non-null   float64 
 14  total_minutes    7983 non-null   float64 
dtypes: bool(1), category(1), float64(9), int64(1), object(3)
memory usage: 1.1+ MB


In [139]:
#checking missing values in merged dataframe
print(" \nTotal number of NaN items present in the columns of the DataFrame 'df_merge': \n\n", df_merge.isnull().sum())

 
Total number of NaN items present in the columns of the DataFrame 'df_merge': 

 order_id              0
store_branch_id       0
store_lat             0
delivery_lat          0
store_long            0
delivery_long         0
shopper_id            0
on_demand             0
seniority             0
order_size            0
found_rate          199
picking_speed         0
accepted_rate        46
rating              162
total_minutes      1995
dtype: int64


As stated before, the "total_minutes" label will be set apart to be used in the preditions phase, so it won't have any new values imputed. For the rest of features ('found_rate', 'accepted_rate' and 'rating '), all missing values will be imputed by using their respectives average values in the series.

In [140]:
# calculating the averages
avg_found = df_merge['found_rate'].mean(axis=0)
avg_accept = df_merge['accepted_rate'].mean(axis=0)
avg_rate = df_merge['rating'].mean(axis=0)
print("Average of found_rate:", avg_found )
print("Average of accepted_rate:", avg_accept )
print("Average of rating:", avg_rate )


Average of found_rate: 0.8633177727784027
Average of accepted_rate: 0.9170133784336325
Average of rating: 4.849341890790547


In [141]:
# replacing the NaN with the average values
df_merge['found_rate'].replace(np.nan, avg_found, inplace=True)
df_merge['accepted_rate'].replace(np.nan, avg_accept, inplace=True)
df_merge['rating'].replace(np.nan, avg_rate, inplace=True)

In [142]:
print(" \nTotal number of NaN items present in the columns of the DataFrame 'df_merge': \n\n", df_merge.isnull().sum())

 
Total number of NaN items present in the columns of the DataFrame 'df_merge': 

 order_id              0
store_branch_id       0
store_lat             0
delivery_lat          0
store_long            0
delivery_long         0
shopper_id            0
on_demand             0
seniority             0
order_size            0
found_rate            0
picking_speed         0
accepted_rate         0
rating                0
total_minutes      1995
dtype: int64


NOTE: All missing values removed from the desired the features

### 3.2 Feature transformation

Geopy library can calculate straight distances between two points using only their geographic coordinates. First, let's change the latitude and longitude values and convert it into a single geographic point using Geopy's `Point` function.

In [143]:
df_merge['store_coord'] = df_merge.apply(lambda row: Point(latitude=row['store_lat'], longitude=row['store_long']), axis=1)
df_merge['delivery_coord'] = df_merge.apply(lambda row: Point(latitude=row['delivery_lat'], longitude=row['delivery_long']), axis=1)
df_merge.sample(1)

,order_id,store_branch_id,store_lat,delivery_lat,store_long,delivery_long,shopper_id,on_demand,seniority,order_size,found_rate,picking_speed,accepted_rate,rating,total_minutes,store_coord,delivery_coord
64,4a331342c470ae118d7d7cafc13fc031,65ded5353c5ee48d0b7d48c591b8f430,-33.48528,-33.497835,-70.57925,-70.59717,961fe1b96546d3cf6ed463424998d60d,False,M,22,0.8459,1.0,0.84,4.6,71.339106,"33 29m 7.008s S, 70 34m 45.3s W","33 29m 52.206s S, 70 35m 49.8116s W"


Finally, using Geopy's `Distance` function, we can obtain the straight line distance (km) between two given points.

In [144]:
#calculating the distance between the store and the delivery point, using their coordinates
df_merge['distance_km'] = df_merge.apply(lambda row: distance(row['delivery_coord'], row['store_coord']).km , axis=1)

In [145]:
#removing coordinate data and renaming the newly found distance
df_merge.drop(['delivery_coord', 'store_coord'], axis=1, inplace=True)
df_merge = df_merge.reindex(['order_id','store_branch_id','shopper_id','store_lat', 'store_long', 'delivery_lat', 'delivery_long','on_demand','seniority', 'order_size', 'found_rate','picking_speed','accepted_rate','rating','distance_km','total_minutes'], axis=1)
df_merge.sample(2)

,order_id,store_branch_id,shopper_id,store_lat,store_long,delivery_lat,delivery_long,on_demand,seniority,order_size,found_rate,picking_speed,accepted_rate,rating,distance_km,total_minutes
8344,f1d198aec327be0ab76e0adb99dca6ed,06fe1c234519f6812fc4c1baae25d6af,8b5eea8a56aafc6cc2dab61ac6a7d690,-33.359768,-70.543273,-33.389837,-70.550349,False,T,3,0.8583,1.14,0.96,4.80,3.399377,37.205612
8783,bbff2f851faf5cca32b57956a1eb973c,fc221309746013ac554571fbd180e1c8,0a9319ad4705c235395ce4c3ef390264,-33.025860,-71.547009,-33.021359,-71.545392,True,T,16,0.8523,2.17,0.92,4.96,0.521546,56.872293


One last transformation to be carried out is the "on_demand" feature, which is currently a boolean. Since booleans aren't good matchs when when working with machine learning models, we'll simply replace "True"=1 and "False"=0

In [146]:
df_merge['on_demand'].dtypes

dtype('bool')

In [147]:
# True = 1, False = 0
df_merge['on_demand'].replace(True,1, inplace=True)
df_merge['on_demand'].replace(False,0, inplace=True)

In [148]:
df_merge.sample(4)

,order_id,store_branch_id,shopper_id,store_lat,store_long,delivery_lat,delivery_long,on_demand,seniority,order_size,found_rate,picking_speed,accepted_rate,rating,distance_km,total_minutes
9598,96e8817a2d87dc5cddbf4c962dff21f5,872488f88d1b2db54d55bc8bba2fad1b,a5b97606d7c120b133204e011464f5ec,-33.347645,-70.542229,-33.329951,-70.529046,0,I,1,0.9012,1.85,0.92,4.80,2.314603,27.778176
3636,5dbc2478c092e95309a9dba91c424d78,1f0e3dad99908345f7439f8ffabdffc4,101d73a2412f17aea66d436a91985984,-33.386547,-70.568075,-33.415490,-70.599249,1,I,17,0.8950,0.83,0.96,4.68,4.326041,65.943577
7952,82682990fec002168fe409039cbd6ba4,c9f0f895fb98ab9159f51fd0297e236d,4fcfb9e0669fb61e72eeccec0d852c1d,-33.401660,-70.514666,-33.392334,-70.496020,1,I,18,0.8893,1.45,0.88,5.00,2.019661,88.972355
6163,bdff1d6ee72e06edb915fdd8c59074cc,45c48cce2e2d7fbdea1afc51c7c6ad26,3e802483b05f016df2f75bd7cac807e8,-33.431283,-70.578740,-33.437039,-70.572044,0,I,19,0.8774,1.00,0.96,4.88,0.891893,68.012361


# 4 Exporting datasets

4.1 Exporting test dataset ("total_minutes" = NaN) using `pd.isna`

In [149]:
# pd.isna returns only a dataframe with the missing values on the "total_minutes" column
df_TEST = df_merge.loc[(pd.isna(df_merge['total_minutes']))]
df_TEST.to_csv('df_TEST.csv')

4.2 Exporting exploration/training dataset ("total_minutes" != NaN) using `pd.notna`

In [150]:
# pd.notna returns only a dataframe with the non-missing values on the "total_minutes" column
df_TRAIN = df_merge.loc[(pd.notna(df_merge['total_minutes']))]
df_TRAIN.to_csv('df_TRAIN.csv')

In [151]:
#final dataset before EDA and ML modeling
df_TRAIN.sample(5)

,order_id,store_branch_id,shopper_id,store_lat,store_long,delivery_lat,delivery_long,on_demand,seniority,order_size,found_rate,picking_speed,accepted_rate,rating,distance_km,total_minutes
1326,59df4367e892d14b88e88fe0c118d004,1679091c5a880faf6fb5e6087eb1b2dc,e8d873c1a370459c3bd51579c3026863,-33.370765,-70.51242,-33.370278,-70.500299,1,T,3,0.8257,2.00,0.92,4.80,1.129251,41.180260
6122,a37ba37f55bc95109ed409ff4720c401,45c48cce2e2d7fbdea1afc51c7c6ad26,76b3b86b29bfabcad7cd369807cb7ced,-33.431283,-70.57874,-33.425354,-70.576772,0,I,44,0.8830,1.22,1.00,4.72,0.682544,122.614332
63,a2920c53ff3059a248b5aa0204dabd46,65ded5353c5ee48d0b7d48c591b8f430,5b8c772bd7e08de3f95da37c7ebbe496,-33.485280,-70.57925,-33.487654,-70.576299,0,I,22,0.8322,1.26,0.92,4.96,0.380190,104.492165
6143,6196256bd60f5ba3cc3b8a6ed571dc8a,45c48cce2e2d7fbdea1afc51c7c6ad26,14303f0f4fdd747547cca3e20afe6e21,-33.431283,-70.57874,-33.417200,-70.589653,0,I,27,0.8442,1.11,0.96,4.88,1.862658,130.347167
6202,61e0ff6ca083ea2b2e3cd3b50853d16a,45c48cce2e2d7fbdea1afc51c7c6ad26,5c3637ccdb35ab78f304df6161d94464,-33.431283,-70.57874,-33.422589,-70.575158,0,M,4,0.8672,3.50,0.92,4.92,1.020151,94.023772
